In [17]:
import pandas as pd
from pandas import read_excel

file_name = 'no_training.xlsx' 
target_data = read_excel(file_name)
target_data = target_data.fillna(0)

In [18]:
feature_cols = list(target_data.columns[1:-7])

X_all = target_data[feature_cols]

In [19]:
from sklearn.ensemble import RandomForestClassifier
import pickle

ccmodel = pickle.load(open('ccmodel.pkl', 'rb'))

prediction = ccmodel.predict(X_all)
sales_cols = list(target_data.columns[:-7])
result = target_data[sales_cols]
result["cc_sales_prediction"] = prediction
result = result[result.cc_sales_prediction == 1]
print(result.shape[0])
print(result[:1000])
result.to_excel("cc_sales_prediction.xlsx", index=False)

43
     Client  Sex  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  \
10       19  1.0   46      94         1       1.0       0.0        0.0   
30       84  0.0   66     235         2       0.0       0.0        0.0   
39      104  0.0   10      65         1       1.0       1.0        0.0   
58      145  0.0   14      52         1       1.0       0.0        0.0   
76      180  1.0   56     174         1       0.0       0.0        0.0   
80      197  0.0   70      58         1       1.0       4.0        0.0   
89      217  0.0   17     152         1       1.0       0.0        0.0   
137     331  1.0   43     160         1       0.0       0.0        0.0   
142     340  1.0   76      80         1       1.0       0.0        0.0   
147     352  0.0   38     194         1       1.0       0.0        0.0   
164     382  0.0   29     150         1       1.0       0.0        0.0   
167     389  0.0   43     158         1       1.0      28.0        0.0   
177     411  0.0   74      33      

/home/syanakieva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [20]:
result

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,cc_sales_prediction
10,19,1.0,46,94,1,1.0,0.0,0.0,0.0,0.0,...,148.446786,0.000000,0.000000,144.125357,5.0,5.0,0.0,0.0,4.0,1
30,84,0.0,66,235,2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1
39,104,0.0,10,65,1,1.0,1.0,0.0,0.0,0.0,...,556.107500,0.000000,53.000000,495.857500,10.0,10.0,0.0,1.0,8.0,1
58,145,0.0,14,52,1,1.0,0.0,0.0,0.0,0.0,...,1242.789286,585.714286,116.039286,539.607143,21.0,21.0,6.0,6.0,8.0,1
76,180,1.0,56,174,1,0.0,0.0,0.0,0.0,0.0,...,473.821429,464.285714,0.000000,7.571429,10.0,10.0,4.0,0.0,1.0,1
80,197,0.0,70,58,1,1.0,4.0,0.0,0.0,0.0,...,30910.964286,0.000000,0.000000,2241.357143,11.0,11.0,0.0,0.0,3.0,1
89,217,0.0,17,152,1,1.0,0.0,0.0,0.0,0.0,...,11430.357143,0.000000,0.000000,11428.571429,2.0,2.0,0.0,0.0,1.0,1
137,331,1.0,43,160,1,0.0,0.0,0.0,0.0,0.0,...,729.190000,471.428571,249.011429,0.000000,36.0,36.0,13.0,16.0,0.0,1
142,340,1.0,76,80,1,1.0,0.0,0.0,0.0,0.0,...,52.642857,0.000000,49.607143,0.000000,3.0,3.0,0.0,2.0,0.0,1
147,352,0.0,38,194,1,1.0,0.0,0.0,0.0,0.0,...,424.226071,142.857143,196.690357,81.285714,14.0,14.0,1.0,8.0,4.0,1
